In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import json
import os
import re
from flashtext import KeywordProcessor

def execute_query_safely(sql, con):
    cur = con.cursor()
    
    # try to execute the query
    try:
        cur.execute(sql)
    except:
        # if an exception, rollback, rethrow the exception - finally closes the connection
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    
    return

path = 'C:/Users/hp/Desktop/'

con = psycopg2.connect(dbname="mimic", user="postgres", password="postgres", host="localhost")

query_schema = 'SET SEARCH_PATH TO public,' + 'mimiciii' + ';'

In [5]:
query = query_schema + '''
SELECT subject_id,hadm_id,text 
FROM noteevents
ORDER BY
subject_id
LIMIT 5000
'''
df8 = pd.read_sql_query(query, con,)
df8

,subject_id,hadm_id,text
0,2,163353.0,Nursing Transfer note\n\n\nPt admitted to NICU...
1,2,163353.0,Neonatology Attending Triage Note\n\nBaby [**N...
2,3,145834.0,Admission Date: [**2101-10-20**] Discharg...
3,3,145834.0,PATIENT/TEST INFORMATION:\nIndication: S/P Car...
4,3,145834.0,PATIENT/TEST INFORMATION:\nIndication: Pericar...
5,3,145834.0,PATIENT/TEST INFORMATION:\nIndication: Left ve...
6,3,NaN,Sinus rhythm. A-V conduction delay. Slight Q-T...
7,3,NaN,Sinus rhythm. First degree A-V block. Low QRS ...
8,3,NaN,\n\n\n
9,3,145834.0,Sinus rhythm\nP-R interval increased\nLate R w...


In [7]:
row_count = df8.shape[0]
with open(path+'symptoms.json') as f:
    j_son = json.load(f)
keyword_processor = KeywordProcessor()
keyword_processor.add_keywords_from_list(j_son)

with open('symptoms_list.csv','w') as f:
    f.write("hadm_id,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6,symptom7,symptom8,symptom9,symptom10,symptom11,symptom12,symptom13,symptom14,symptom15,symptom16,symptom17,symptom18,symptom19,symptom20,symptom21,symptom22,symptom23,symptom24,symptom25,symptom26,symptom27\n")
    for x in range(0,row_count+1):
        list1 = []
        text1 = str(df8.values[x]).split(".")
        h_admission_id = str(df8['hadm_id'].iloc[x])
        if h_admission_id == 'NaN' or h_admission_id == 'nan' or h_admission_id == '':
            continue
        else:
            hadm_id = h_admission_id + ','
            for i in text1:
                if(re.findall("no .*?",i) or re.findall("did not exhibit .*?",i) or re.findall("did not report .*?",i) or re.findall("did not show symptoms of .*?",i) or re.findall("negative .*?",i) or re.findall("without .*?",i)):
                    continue
                else:
                    temp = keyword_processor.extract_keywords(i)
                    if len(temp) != 0:
                        for  i in temp:
                            list1.append(str(i))
            data1 = sorted(list(set(list1)))
            if len(data1) == 0:
                continue
            elif len(data1) < 26:
                f.write(hadm_id)
                f.write(",".join(data1))
                f.write("\n")

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['heart failure']
[]
['diabetes']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['cellulitis']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['heart failure']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['tachycardia']
[]
[]
[]
[]
[]
[]
[]
[]
['chest pain']
[]
[]
['heart failure']
[]
[]
[]
[]
['cellulitis']
[]
[]
[]
[]
[]
['wheezing', 'asthma']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['heart failure']
[]
['diabetes']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

In [ ]:
df = pd.read_csv('symptoms_list.csv', error_bad_lines=False)
df